이 Python 3 environment는 분석에 유용한 여러 libraires들을 이용합니다.
이 environment는 kaggle/python docker image로 만들어져 있습니다. : https://github.com/kaggle/docker-python

일단 먼저 예시로 몇가지 유용한 packages들을 소개드리겠습니다.

In [ ]:
import numpy as np # linear algebra (선형대수학)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) (데이터 처리, CSV file의 I/O)

# Input data files는 "../input/" directory에 있습니다.
# 이것을 실행시키면 (run을 누르거나, Shift+Enter를 누르세요) input directory에 있는 file들을 보여줄 것입니다.
# 그리고 현재 directory에 write한 모든 results들은 output 형태로 저장될 것입니다.

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

이 kernel은 Keras를 이용해 CNN을 구현하고 싶어하는 초심자들을 위한 것입니다. 이 kernel과 함께라면, 당신은 좋은 점수를 받을 수 있고, Keras에 대해서도 배울 수 있습니다.
Keras는 model을 initialize하고 우리가 원하는 layer를 쌓아갈 수 있는 간단한 frameworks 입니다. Deep neural networks를 아주 쉽게 만들 수 있게 도와주죠.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [ ]:
#Load the data.
train = pd.read_json("../input/train.json")

In [ ]:
test = pd.read_json("../input/test.json")

### 데이터에 대한 기본 설명

Sentinel 인공위성은 지구에서 680Km 떨어진 지점에 위치해 있습니다. 특정 입사각에서 pulses of signal들을 보낸 후, 돌아오는 신호를 기록합니다. 반사(산란)되어 되돌아온 신호는  "backscatter"라고 부릅니다. 우리에게 주어진 데이터는 backscatter 데이터입니다. 일반적인 backscatter 형태를 띄는데, 그 형태는 다음과 같습니다 : 

$σo (dB) = βo (dB) + 10log10 [sin(ip) / sin (ic)] $

1. ip = 특정 pixel에서의 입사각
2. ic = 이미지 중심에서의 입사각
3. K = 상수

우리는 이 $σo$ 값을 갖고 있습니다.

### $σo$의 특징
기본적으로 $σo$는 signal이 산란된 표면에 따라 다릅니다. 예를 들어, 어떠한 입사각에 대해 다양한 표면의 종류에 따라 데이터를 뽑아 보겠습니다. 데이터를 보면, 표면의 종류에 따라(Water, Settlements, Agriculture, Barren) 다른 값을 나타내는 것을 볼 수 있습니다.

*             WATER...........           SETTLEMENTS........           AGRICULTURE...........          BARREN........

1.**HH:**     -27.001   ................                     2.70252       .................                -12.7952        ................    -17.25790909

2.**HV: **      -28.035      ................            -20.2665             ..................          -21.4471       .................     -20.019

배에서 산란되어 되돌아온 signal의 data는 나와있지 않지만, 배와 얼음은 다른 물체이기 때문에 둘의 값은 완전히 달라야 할 것입니다. 우리는 이것을 이용해 배와 얼음을 구분할 수 있을 것입니다.

### HH와 HV는 무엇입니까?
이 Sentinel 인공위성은 RISTSAT 인공위성 (인도의 원격탐사 인공위성)과 같습니다. 이들은 pulses of signal을 보낼 때, 수평으로 편광(H polarization)된 signal을 보냅니다. **따라서 signal에 수직 성분(V polarization)은 포함되어 있지 않습니다.**.  보낸 H-pings(H poloarized 된 signal)들이 물체에 부딪히게 되면, 산란되어 편광상태가 달라지게 됩니다. 따라서 수평 성분(H) 뿐이었던 signal이 수평 성분(H)과 수직 성분(V)의 합(HH or HV)으로 반환됩니다.

그런데 HH, HV Signal은 있는데 왜 VV Signal은 없을까요? 수평으로 편광(H polarization)된 signal은 절대 VV signal이 될 수 없기 때문입니다.

### feature 만들기
그럼 이제 feature를 봅시다. 우리는 주어진 HH, HV data를 보통 CNN에서 사용되는 RGB 즉 3-channel 데이터처럼 만들 것 입니다. 3-channel 데이터를 만들기 위해, HH, HV, 그리고 HH-HV의 평균값을 3rd channel으로 사용할 것입니다.

In [ ]:
# training 데이터를 만듭니다.
# HH, HV, HH와 HV의 평균 총 3개의 band를 만듭니다. 
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [ ]:
#빙산을 봅시다
#Take a look at a iceberg
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):
    print(c.shape)
    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
plotmy3d(X_band_1[12,:,:], 'iceberg')

데이터를 plot해보니 멋진 빙산이 보이네요. 레이더 데이터를 이용해서 빙산을 만들게 되면, 여기서 보이는 것과 같은 산 모양을 나타낼 것입니다. 왜냐하면, 이것은 실제 이미지 데이터가 아니라, 레이더에서 산란된 데이터를 갖고 만든 데이터이기 때문에 뾰족한 봉우리 모양을 갖도록 왜곡됩니다. 그리고 배의 모양은 점, 혹은 연장된 점으로 왜곡될 것입니다. 어쨋거나 우리는 여기서 빙산과 배의 구조적인 차이점이 있음을 알 수 있고, CNN을 이용해 이 차이점을 바탕으로 빙산과 배를 구분하도록 할 것입니다. 우리가 레이더로부터 얻은 backscatter를 이용하여 합성 사진을 만들 수 있다면, 많은 도움이 될 것입니다.  

In [ ]:
plotmy3d(X_band_1[14,:,:], 'Ship')

이것은 배입니다, 연장되어 있는 점같이 생겼네요. 그런데 이 레이더 데이터만 갖고 배 모양을 시각화하기엔 해상도가 부족합니다. 그리고 그것을 도와주기위해 CNN이 존재합니다. 배-빙산 classification과 관련된 참고해 볼만한 논문이 있습니다.
http://elib.dlr.de/99079/2/2016_BENTES_Frost_Velotto_Tings_EUSAR_FP.pdf
그런데 그들은 훨씬 좋은 해상도를 갖고 있는 데이터를 사용하였기 때문에, 그들이 이용한 CNN은 여기에 맞지 않을 것 같습니다.

### CNN Layer 만들기
자 이제 Keras로 CNN을 만들어 봅시다. Keras는 다른 framework보다 훨씬 좋습니다. 아마 당신도 분명히 Keras를 좋아할 거에요.

In [ ]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [ ]:
target_train=train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.75)

In [ ]:
#Without denoising, core features.
import os
gmodel=getModel()
gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=50,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

이 LB 점수는 조금 다르게 나올 수도 있는데, 저는 0.210를 받았습니다.

In [ ]:
gmodel.load_weights(filepath=file_path)
score = gmodel.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
predicted_test=gmodel.predict_proba(X_test)

In [ ]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

### 결론
점수를 올리기 위해 추가적으로 Speckle filtering, Indicence angle normalization, 그리고 다른 preprocessing을 사용해 보았는데 잘 작동하지 않았습니다. 원한다면 그것들을 해보는 것도 괜찮겠지만, 저같은 경우 좋은 결과는 나오지 않았습니다.

어짜피 이 kernel을 이용한다고 해서 상위 10명 안에 들지는 못할것이니, 좋은 정보를 하나 드리겠습니다. Test dataset에 8000개의 이미지가 존재하는데, 그것을 활용해 보세요. pseudo labeling을 해서 성능을 높혀볼 수 있을 것입니다. 관련 링크 첨부합니다:
https://towardsdatascience.com/simple-explanation-of-semi-supervised-learning-and-pseudo-labeling-c2218e8c769b

이 kernel이 도움이 되었다면 Upvote해주세요 :)